In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob

In [ ]:
#in future, replace with list of files from error info csv file

flist=glob.glob('logs/DAQ_logs/lc-ASIC_tsalgo_randdata_17mar_16*csv')
flist.sort()
oot_flist=glob.glob('logs/DAQ_logs/lc-ASIC_tsalgo_randdata_oot_17mar_16*csv')
oot_flist.sort()


In [ ]:
# fig,ax=plt.subplots(2,1,figsize=(10,16))
fig,ax=plt.subplots(3,1,figsize=(10,24))
oot_errCount=[]
oot_timestamps=[]
for fName in flist[:]:
    t='22'+''.join(fName[:-4].split('_')[-2:])
    t=pd.to_datetime(t,format='%y%d%b%H%M%S')
    df_ASIC=pd.read_csv(fName)
    df_em=pd.read_csv(fName.replace('ASIC','emulator'))
    oot_timestamps.append(t)
    oot_errCount.append((~(df_ASIC==df_em).all(axis=1)).sum())

    
    t=(np.arange(4095))*.025
    ax[0].plot(t,(df_ASIC==df_em).all(axis=1).astype(int).values)
    # plt.xlim(440,470)
    ax[0].set_xlim(0,2000*.025)
    ax[0].set_xlabel('Time (µs)')
    ax[0].set_ylabel('Match (1) / Mismatch (0)')

    bits_ASIC=np.array([[*f"{int(''.join(x),16):0416b}"] for x in df_ASIC.values],dtype=int)
    bits_em=np.array([[*f"{int(''.join(x),16):0416b}"] for x in df_em.values],dtype=int)
    x=(416-(bits_ASIC==bits_em).sum(axis=1))
    ax[1].plot(x)
    ax[1].set_xlim(0,2000)
    ax[1].set_xlabel('BX Number')
    ax[1].set_ylabel('Bit errors per BX (out of 416 total)')
    
ax[0].patch.set_alpha(1.)
ax[1].patch.set_alpha(1.)
ax[2].plot(oot_timestamps,oot_errCount)
fig.patch.set_alpha(1.)

In [ ]:
# fig,ax=plt.subplots(2,1,figsize=(10,16))
fig,ax=plt.subplots(3,1,figsize=(10,24))
oot_errCount=[]
oot_timestamps=[]
for fName in oot_flist[:]:
    t='22'+''.join(fName[:-4].split('_')[-2:])
    t=pd.to_datetime(t,format='%y%d%b%H%M%S')
    df_ASIC=pd.read_csv(fName)
    df_em=pd.read_csv(fName.replace('ASIC','emulator'))
    oot_timestamps.append(t)
    oot_errCount.append((~(df_ASIC==df_em).all(axis=1)).sum())

    
    t=(np.arange(4095))*.025
    ax[0].plot(t,(df_ASIC==df_em).all(axis=1).astype(int).values)
    # plt.xlim(440,470)
    ax[0].set_xlim(0,2000*.025)
    ax[0].set_xlabel('Time (µs)')
    ax[0].set_ylabel('Match (1) / Mismatch (0)')

    bits_ASIC=np.array([[*f"{int(''.join(x),16):0416b}"] for x in df_ASIC.values],dtype=int)
    bits_em=np.array([[*f"{int(''.join(x),16):0416b}"] for x in df_em.values],dtype=int)
    x=(416-(bits_ASIC==bits_em).sum(axis=1))
    ax[1].plot(x)
    ax[1].set_xlim(0,2000)
    ax[1].set_xlabel('BX Number')
    ax[1].set_ylabel('Bit errors per BX (out of 416 total)')
    
ax[0].patch.set_alpha(1.)
ax[1].patch.set_alpha(1.)
ax[2].plot(oot_timestamps,oot_errCount)
fig.patch.set_alpha(1.)

In [ ]:
len(flist)

In [ ]:
oot_flist

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(15,6))

fName='logs/DAQ_logs/lc-ASIC_tsalgo_randdata_oot_17mar_153415.csv'
df_ASIC=pd.read_csv(fName)
df_em=pd.read_csv(fName.replace('ASIC','emulator'))

bx=range(1,1000)
bits_ASIC=np.array([[*f"{int(''.join(x),16):0416b}"] for x in df_ASIC.values],dtype=int)
bits_em=np.array([[*f"{int(''.join(x),16):0416b}"] for x in df_em.values],dtype=int)
for j in range(8):
    eTx0_ASIC=bits_ASIC[:,j*32:(j+1)*32][::-1].flatten()[::-1]
    eTx0_em=bits_em[:,j*32:(j+1)*32][::-1].flatten()[::-1]

    y=[]
    y2=[]
    p=8
    for k in bx:
        n=k*32
        a=eTx0_em[n:n+64]
        b=np.array([eTx0_ASIC[n+i-p:n+64+i-p] for i in range(2*p)])
        y.append((a==b).sum(axis=1).argmax())
        y2.append((a==b).sum(axis=1).max())
    #     print(j,(a==b).sum(axis=1).argmax(),(a==b).sum(axis=1).max())
    y=np.array(y)-p
    ax[0].plot(bx,y,label=f'eTx {j}')
    ax[1].plot(bx,y2)
ax[0].set_ylim(-8,8)
ax[0].set_xlabel('BX')
ax[1].set_xlabel('BX')
ax[0].set_ylabel('Best Bit Shift')
ax[1].set_ylabel('Correct Bits at best bit shift')
ax[0].legend()